In [1]:
import pandas as pd
import time

from datetime import datetime, timedelta
from google_play_scraper import Sort, reviews

In [2]:
# get all reviews ranging from today to first instance of day before yesterday
# drop columns 
# rename columns
# filter rows - remove reviews from today and reviews from day before yesterday

# insert to database

In [3]:
def get_extraction_is_complete(data):
    today = datetime.today()
    today = pd.Timestamp(today).floor('d')
    yesterday = today - pd.Timedelta(days=1)
    
    extraction_is_complete = data['at'].apply(lambda ts: ts < yesterday).any()
    return extraction_is_complete


def get_continue_extracting_reviews(data):
    continue_extracting_reviews = not(get_extraction_is_complete(data))
    return continue_extracting_reviews


def extract_next_batch(continuation_token, app_id):
    results, continuation_token = reviews(
        app_id,
        continuation_token=continuation_token
    )
    
    next_batch = pd.DataFrame(results)
    return next_batch, continuation_token


def extract_review_data(app_id='com.tgc.sky.android'):
    results, continuation_token = reviews(
        app_id,
        sort=Sort.NEWEST,
        count=200
    ) 
    review_data = pd.DataFrame(results)
    
    continue_extracting_reviews = get_continue_extracting_reviews(review_data)
    while(continue_extracting_reviews):
        next_batch, continuation_token = extract_next_batch(continuation_token, app_id=app_id)
        continue_extracting_reviews = get_continue_extracting_reviews(next_batch) 
        review_data = review_data.append(next_batch, ignore_index=True)
        time.sleep(1)
            
    return review_data

In [4]:
review_data = extract_review_data()

In [6]:
review_data

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,gp:AOqpTOEjBUJXK1alBzwYKT7OFhFXg2AKLVvd_0TRtp9...,Emil Noel Dulnuan,https://play-lh.googleusercontent.com/a-/AOh14...,Awesome,5,0,None,2021-04-28 20:46:56,None,None
1,gp:AOqpTOGKYZ74nifldtQZkw4PPa9kT3klvByAqXHZ2fm...,EASY GAMING,https://play-lh.googleusercontent.com/a-/AOh14...,this is one of game make me smile when I play it,5,0,0.13.3 (165938),2021-04-28 20:40:02,None,None
2,gp:AOqpTOGzThYEPr1NrONLvTA3gc5YazDVEB4UXaz_7Xh...,8A32 Parimal Ingle,https://play-lh.googleusercontent.com/a-/AOh14...,relaxes my mind and me by its graphics and its...,5,0,0.13.3 (165938),2021-04-28 20:22:14,None,None
3,gp:AOqpTOGhfKMDsLv1-UACnfjmYwFhQ7X93Tl-_9MX1NJ...,Jemmillee Calayag,https://play-lh.googleusercontent.com/a/AATXAJ...,☺🤩😍,1,0,0.13.3 (165938),2021-04-28 19:14:33,None,None
4,gp:AOqpTOGhco2hi3-NXIp3YxKSRp3a3fG9F4xOAwGq7QK...,Samantha Gacha,https://play-lh.googleusercontent.com/a-/AOh14...,very nice game the music is good too!,5,0,0.13.3 (165938),2021-04-28 19:11:02,None,None
...,...,...,...,...,...,...,...,...,...,...
195,gp:AOqpTOHB97OQdam8YhngeWKMCAVaM13tAIwyKBUFCjY...,Ban Plays,https://play-lh.googleusercontent.com/a-/AOh14...,Boring.,1,1,None,2021-04-25 22:08:45,None,None
196,gp:AOqpTOFtpsZJm5DETcGhD94Fg4dEdQKr_LY_DcAX9BI...,TheKillerBoy,https://play-lh.googleusercontent.com/a-/AOh14...,Very peaceful game and relaxing and has abit o...,5,0,0.13.3 (165938),2021-04-25 21:30:15,None,None
197,gp:AOqpTOGzIEhZhCmjVqPPg9CaShFkivzX6EIHt_k8BCs...,Emperor Crimson,https://play-lh.googleusercontent.com/a-/AOh14...,Its really great,5,0,0.13.3 (165938),2021-04-25 21:08:21,None,None
198,gp:AOqpTOGGigOqiq93-6Tp36KKNRiVBfiutT_37aUd5TH...,SaveshGaming,https://play-lh.googleusercontent.com/a-/AOh14...,This Game is Good,4,0,None,2021-04-25 20:54:31,None,None
